In [ ]:
import requests
import json
import pandas as pd
import datetime
from IPython.display import display, clear_output

In [ ]:
url = 'https://data.binance.com/api/v3/klines'
symbol = 'ETHUSDT'
interval = '1h

data = []
STARTDATE = datetime.datetime(2022, 7, 1, tzinfo=datetime.timezone.utc)
ENDDATE = datetime.datetime.utcnow().replace(hour = 0, minute = 0,second = 0,microsecond=0, tzinfo=datetime.timezone.utc) + datetime.timedelta(1)
dates = pd.date_range(STARTDATE, ENDDATE, freq = '4H')
for start, end in zip(dates[:-1], dates[1:]):
    clear_output(wait=True)
    display(f"Start: {start.strftime('%Y-%m-%d %H:%M:%S')}, End: {end.strftime('%Y-%m-%d %H:%M:%S')}")
    par = {'symbol': symbol, 'interval': interval, 'startTime': str(int(start.timestamp()*1000)), 'endTime': str(int(end.timestamp()*1000))}
    #retry if we cannot successfully fetch the data from start to end.
    while True:
      res = requests.get(url, params= par)
      if res.status_code == 200:
          data += json.loads(res.text)
          break
      else:
          print("retry")
df = pd.DataFrame(data)
#format columns name
df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
df.index = [datetime.datetime.utcfromtimestamp(x/1000.0) for x in df.datetime]
df=df.astype(float)
df.drop_duplicates(inplace = True)
df.sort_index(inplace = True)
df['Datetime'] = df['datetime'].apply(lambda x: datetime.datetime.utcfromtimestamp(x/1000.0)).shift(-1)
df['timestamp'] = df['datetime'].shift(-1)/1000
df_summary = df[['Datetime', 'timestamp', 'close']].set_index('Datetime').head(-1)